In [1]:
import psycopg2
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sqlalchemy as sa
import numpy as np
import math
import Orange
import cv2

initialize a Database, giving the username, password, server, port and the database name. Returns engine variable type

### 2.1 deteçao de mitosis com dataset

In [2]:
def initializeDb(username, password, server, port, database):
    parameters = { 
               'username': username, 
               'password': password,
               'server':   server,
                'port' : port,
               'database': database
             }
    connection = 'postgresql://{username}:{password}@{server}:{port}/{database}'.format(**parameters);
    
    return sa.create_engine(connection, encoding="utf-8")

- Adds new features to the given dataframe :
 - pctarea - change in percentage of area comparing with previous frame 
 - pctperimete - change in percentage of perimeter comparing with previous frame 
 - pctcircularity - change in percentage of the circularity comparing with previous frame 

In [3]:
def calculateNewFeatures(features):
    features['pctarea'] = features[['id','area','areadiff','perimeter','circularity']].pct_change()['area']*100
    features['pctperimeter'] = features[['id','area','areadiff','perimeter','circularity']].pct_change()['perimeter']*100
    features['pctcircularity'] = features[['id','area','areadiff','perimeter','circularity']].pct_change(periods=1)['circularity']*100
    for index, row in features.iterrows():
        if math.isnan(row['areadiff']):
            features.loc[index,'pctarea'] = float('nan')
            features.loc[index,'pctperimeter'] = float('nan')
            features.loc[index,'pctcircularity'] = float('nan')


- Procedure that calculates the lowest or Highest change in a procedure with the previous frames
- 1st for cicle creates the needed columns names for the auxiliary dataframe
- 2nd for cicle calculates the change in percentage comparing needed frames
- 3rd for cicle set none values to incalculate values
 - frames : defines the required number of previous frames to compare with ( 5 by default )
 - minimun : true to get the lowest change, false for the highest ( true by default )
 - feature : feature selected to study
 
 

In [4]:
def calculatePctChange(features,feature, frames=5, minimun=True):
    columns = []
    for i in range(1,frames):
        columns.append("a"+str(i+1));
    d = pd.DataFrame(columns = columns)
    for i in range(1,frames):
        d["a"+str(i+1)]= features[['id','area','areadiff','perimeter','circularity']].pct_change(periods=i+1)[feature]*100
    d['pct'+feature] = features['pct'+feature];
    for index, row in d.iterrows():
        if math.isnan(row['pct'+feature]):
            for i in range(1,frames):
                d.loc[index,"a"+str(i+1)] = float('nan')
            d.loc[index+1,"a2"] = float('nan')
        for i in range(2,frames):
            if math.isnan(row["a"+str(i)]):
                d.loc[index+1,"a"+str(i+1)] = float('nan')
    if minimun:
        features['minPct' + feature] = d.min(axis=1)
    else:
        features['maxPct' + feature] = d.max(axis=1)

Add a column to the elected dataframe that shows if the cell has values in previous frames ( 5 by default)

In [5]:
def checkHistory(features, frames = 5):
    features['historicalRecord'] = True
    for index, row in features.iterrows():
            for i in range (0,frames):
                if index-i>=0:
                    if(features.loc[index]['frame'] - features.loc[index-i]['frame'] != i or features.loc[index]['id'] !=features.loc[index-i]['id']):
                        features.loc[index,'historicalRecord'] = False;
                        break;
                else:
                    features.loc[index,'historicalRecord'] = False;


add 2 new columns that stores if that is the last frame or the one before that one, using the cells table of that database to get the ground truth values

In [6]:
def labelMitosis(engine,features):
    cells_table = pd.read_sql_query('select * from cells',con=engine);
    features['Mitosis'] = False;
    features['Pre-Mitosis'] = False;
    parentid_table = cells_table.groupby(['parentid']).count()
    parentid_table = parentid_table.loc[~(parentid_table.index == 0)]['id']
    tmp = parentid_table.index[:]
    for idx in tmp:
        features.loc[((features['gt_id'] == idx) & (features['frame']  == cells_table.loc[cells_table['id'] == idx].iloc[0].dead-1)),'Pre-Mitosis'] = True
        features.loc[((features['gt_id'] == idx) & (features['frame']  == cells_table.loc[cells_table['id'] == idx].iloc[0].dead)),'Mitosis'] = True

In [7]:
def removeIncompleteCells(features):
    df = features[features.incomplete != True];
    df = df.drop(columns='incomplete');
    df = df.reset_index()
    return df;

In [8]:
def main (username,password,server,port,database,frames=5):
   
    engine = initializeDb(username,password,server,port,database);
    
    features = pd.read_sql_query('select * from features',con=engine);
    
    features = removeIncompleteCells(features);
    
    checkHistory(features);
    
    calculateNewFeatures(features)
    
    calculatePctChange(features,'area');
    
    calculatePctChange(features,'circularity',minimun = False);
    #print features
    #labelMitosis(engine,features);
  
    return features;

In [9]:
features = main("postgres","postgres","localhost",5434,"Fluo-N2DH-SIM+_GT")
#print features.loc(features[pre-mitosis] = True)

In [10]:
print features.loc[features['id'] == 4]

    index  id  frame    area   perimeter  areadiff  perimeterdiff  \
75    169   4     18  2224.0  198.308658     -59.0       4.828427   
76    170   4     19  2084.0  190.509668    -140.0      -7.798990   
77    171   4     20  2110.5  188.752309      26.5      -1.757359   
78    172   4     21  2225.5  192.409163     115.0       3.656854   
79    173   4     22  2207.5  193.379726     -18.0       0.970563   
80    174   4     23  2142.0  186.994949     -65.5      -6.384776   
81    185   4     34  1563.5  167.923882    -435.5     -15.414214   
82    186   4     35  1381.0  160.509668    -182.5      -7.414214   
83    187   4     36  1518.5  170.267027     137.5       9.757359   
84    188   4     37  1372.0  168.166522    -146.5      -2.100505   
85    189   4     38  1088.0  155.338095    -284.0     -12.828427   
86    190   4     39   797.0  117.053824    -291.0     -38.284271   
87    191   4     40   639.0  108.568542    -158.0      -8.485281   
88    192   4     41   654.5  109.

In [11]:
xpto = features.loc[(((features['minPctarea']>20)   | ((features['pctarea']>15) & (features['minPctarea']>5) & (features['pctcircularity']<1) & (features['maxPctcircularity'] < 3))) & (features['pctperimeter']>5) & (features['historicalRecord']==True))]
print xpto[['id','frame','pctarea','pctperimeter', 'pctcircularity','minPctarea','maxPctcircularity','Pre-Mitosis']]

KeyError: "['Pre-Mitosis'] not in index"

## 2.2Associate cells with previous frames

In [12]:
def main2 (username,password,server,port,database):
   
    return initializeDb(username,password,server,port,database);
    
    

In [47]:
engine = main2("postgres","postgres","localhost",5434,"Fluo-N2DL-HeLa_ST")



In [48]:
def createTmpTable(engine,originalTable,tmpTable,col):
    #engine.execute("drop table %s"%tmpTable)
    
    sql = "create table %s as (\
    Select frame,geom from %s)"%(tmpTable,originalTable)
    engine.execute(sql);
    
    sql = "ALTER TABLE %s \
    ADD COLUMN %s bigint ;"%(tmpTable,col)
    engine.execute(sql);
    
    sql = "ALTER TABLE %s ADD COLUMN id SERIAL PRIMARY KEY;"%(tmpTable)
    engine.execute(sql);
    
createTmpTable(engine,"lifetime","lifetmp","myId")

In [49]:
def compareCell(engine,table,frame,id):
    sql = "Select b.myid as cell1,St_area(b.geom) as area1, c.myid as cell2,c.id as idgenerated,St_area(c.geom) as area2,\
            (Select radius from ST_MinimumBoundingRadius(b.geom)),\
            ST_distance(ST_Centroid(b.geom),ST_Centroid(c.geom)) as centDistance,\
            ST_Intersects(ST_MakeValid(c.geom),ST_MakeValid(b.geom)) as l from\
            (select * from %s where id = %s ) as b,\
            (select * from %s where frame = %s) as c"%(table,id,table,frame);
    compare = pd.read_sql_query(sql ,con=engine)
    #return compare.loc[(compare['l']==True) & (compare['radius'] + 5 > compare['centdistance'])]
    return compare

In [50]:
def setInitialIDs(engine,cells_table,GT_table,table):
    cells_table = pd.read_sql_query('select * from %s'%(cells_table),con=engine)
    tmp = pd.read_sql_query('select * from %s'%(GT_table),con=engine)
    for id in cells_table.loc[cells_table['parentid'] == 0]['id']:
        try:
            firstAppear = tmp[['id','frame']].loc[tmp['id']==id].sort_values(by=['frame']).iloc[0];
            sql3 = "select * from %s \
            join %s \
            on %s.geom = %s.geom \
            where %s.frame = %s and %s.frame = %s and id = %s"%(table,GT_table,table,GT_table,GT_table,firstAppear['frame'],table,firstAppear['frame'],firstAppear['id']);
            same = pd.read_sql_query(sql3 ,con=engine)
            engine.execute("UPDATE %s SET myId = %s WHERE id = %s;"%(table,same.cellid[0],same.id[0]));
        except:
            print "cell " + str(id) + " has no representation"

In [51]:
def restartIds(engine,table):
    sql = """ALTER TABLE %s
    drop COLUMN myId ;

    ALTER TABLE %s
    ADD COLUMN myId bigint not null default 0;"""%(table,table)
    engine.execute(sql);
restartIds(engine,"lifetmp");

In [52]:
def calculateNumberOfMitosis(engine):
    cellsGT = pd.read_sql_query('Select * from cells',con=engine);
    mitosis = cellsGT.groupby(['parentid']).count()
    return mitosis.loc[mitosis.id == 2].size/3

In [53]:
def setInitialIDs(engine,table):
    engine.execute("UPDATE %s SET myId = id WHERE frame = 0;"%(table));

In [54]:
def CorrespondCells(engine,table):
    lifeTmp = pd.read_sql_query('select * from %s'%(table),con=engine);
    frames = lifeTmp['frame'].nunique();
    #lastId = lifeTmp['gt_id'].nunique()
    missingVal = [];
    mitosisFrame = [];
    for i in range(1,frames):
        j = [];
        l = lifeTmp.loc[(lifeTmp['frame'] ==  i-1)];
        #print l
        for index,row in l.iterrows():
            j.append(int(row.id))
        #print j
        for a in j:
            compare = compareCell(engine,table,i,a)
            match = compare.loc[(compare['l']==True) & (compare['radius'] + 5 > compare['centdistance'])]
            if len(match.index) == 1:
                engine.execute("UPDATE lifetmp SET myId = %s WHERE id = %s;"%(match.cell1.iloc[0],match.idgenerated.iloc[0]));
            elif len(match.index) == 2:
                if match.cell1.iloc[0]<100:
                    engine.execute("UPDATE lifetmp SET myId = %s WHERE id = %s;"%((match.cell1.iloc[0]+100),match.idgenerated.iloc[0]));
                    engine.execute("UPDATE lifetmp SET myId = %s WHERE id = %s;"%((match.cell1.iloc[1]+200),match.idgenerated.iloc[1]));
                else:
                    engine.execute("UPDATE lifetmp SET myId = %s WHERE id = %s;"%((match.cell1.iloc[0]+10**(len(str(match.cell1.iloc[0]/100))+2)),match.idgenerated.iloc[0]));
                    engine.execute("UPDATE lifetmp SET myId = %s WHERE id = %s;"%((match.cell1.iloc[1]+2*10**(len(str(match.cell1.iloc[1]/100))+2)),match.idgenerated.iloc[1]));
            #quit();
            
            else:
                print(a,i)
                print(len(match.index))
            #quit();
        lifeTmp = pd.read_sql_query('select * from %s'%(table),con=engine);

In [55]:
setInitialIDs(engine,"lifetmp")

In [56]:
CorrespondCells(engine,"lifetmp")

(82, 2)
0
(739, 3)
0
(1456, 4)
0
(2231, 5)
0
(4251, 7)
0
(5415, 8)
0
(148, 12)
0
(201, 13)
0
(266, 14)
0
(316, 15)
0
(374, 15)
0
(393, 16)
0
(567, 18)
0
(794, 21)
0
(397, 22)
0
(806, 22)
0
(1063, 25)
0
(1367, 30)
0
(1474, 31)
0
(1483, 31)
0
(1544, 32)
0
(1685, 34)
0
(1746, 35)
0
(1955, 37)
0
(185, 38)
0
(2096, 39)
0
(2102, 39)
0
(2111, 40)
0
(2123, 40)
0
(2134, 40)
0
(2130, 40)
0
(2122, 40)
0
(2177, 40)
0
(2253, 41)
0
(2328, 42)
0
(2385, 42)
0
(2427, 43)
0
(2480, 43)
0
(2615, 45)
0
(2642, 45)
0
(2663, 45)
0
(2795, 47)
0
(2887, 48)
0
(2950, 48)
0
(3021, 49)
0
(3042, 49)
0
(2979, 49)
0
(3225, 51)
0
(3318, 52)
0
(3319, 52)
0
(3411, 53)
0
(945, 56)
0
(958, 57)
0
(3861, 57)
0
(3864, 57)
0
(4056, 59)
0
(4188, 60)
0
(4458, 62)
0
(4706, 64)
0
(4784, 65)
0
(4822, 65)
0
(5339, 70)
0
(5345, 70)
0
(5686, 73)
0
(5710, 73)
3
(2697, 74)
0
(6034, 76)
0
(6045, 76)
0
(6075, 76)
0
(6100, 77)
0
(6163, 77)
0
(6168, 77)
0
(6329, 79)
0
(6503, 80)
0
(6516, 80)
0
(6532, 80)
0
(6609, 81)
0
(6640, 81)
0
(6896, 8

In [57]:
def programCycle(engine,table):
    sql = '''select myid, inicio, fim,
    (select DISTINCT b.myid iniciopai
    from %s b
    where b.myid = MOD(a.myid,100)) iniciopai
    from
    (select myid, min(frame) inicio, max(frame) fim
    from lifetmp
    group by myid) a
    order by iniciopai asc, inicio desc'''%(table)

    pgCycle = pd.read_sql_query(sql,con=engine);
    #print pgCycle
    return pgCycle


In [58]:
def getProgramLifeCycle(engine,table):
    cycle = programCycle(engine,table)
    tmp = cycle.groupby(['iniciopai']).apply(lambda x: x['fim'].values)
    df = tmp.to_frame()
    df2 = pd.DataFrame( columns=['id2','deaths']);
    columns = list(df2)
    data2 = [];
    for index, row in df.iterrows():
        a = row[0].tolist()
        a.append(cycle.loc[cycle.myid==row.name].inicio.iloc[0])
        a.sort()
        if index > 0:
            values = [index,[int(v) for v in a ]];
            zipped = zip(columns, values)
            a_dictionary = dict(zipped)
            data2.append(a_dictionary)

    df2 = df2.append(data2,True)
    return df2
lifeCycle = getProgramLifeCycle(engine,"lifetmp")

In [59]:
def getAllDeathFrames(id,table,arr):
    arr.append(id)
    sons = table.loc[table['parentid']==id].id.unique();
    if sons.size > 0:
        for a in sons:
            getAllDeathFrames(a,table,arr)
    else:
        return arr
    return arr

In [60]:
def getGTLifeCycle(engine,cellsTable):
    cellsGT = pd.read_sql_query('select * from %s'%(cellsTable),con=engine)
    df = pd.DataFrame( columns=['id','deaths']);
    columns = list(df)
    data = [];
    initCells = cellsGT.loc[cellsGT['parentid']==0].id.unique();

    for cell in initCells:
        family = getAllDeathFrames(cell,cellsGT,[]);
        deaths = cellsGT.loc[cellsGT['id'].isin(family)]['dead'];
        a = deaths.values.tolist();
        a.append(cellsGT.loc[cellsGT['id'] == cell]['born'].iloc[0])
        a.sort();
        values = [cell,[int(v) for v in a ]];
        zipped = zip(columns, values)
        a_dictionary = dict(zipped)
        data.append(a_dictionary)
    df = df.append(data,True)
    return df
GTCycle = getGTLifeCycle(engine,"cells")
#print GTCycle.iloc[7].deaths

In [64]:
def compareValues(dataframe1,dataframe2):
    data = [];
    df = pd.DataFrame( columns=['id','id2','deaths']);
    columns = list(df);
    for index,row in dataframe1.iterrows():
        for index2,row2 in dataframe2.iterrows():
            try:
            #(np.isin(np.array(dataframe1['deaths'][index]),np.array(dataframe2['deaths'][index2]))).all() and len(np.array(dataframe1['deaths'][index]))    
                if (np.array(dataframe1['deaths'][index]) == np.array(dataframe2['deaths'][index2])).all():
                    values = [dataframe1['id'][index],dataframe2['id2'][index2],dataframe1['deaths'][index]]
                    zipped = zip(columns, values)
                    data_dictionary = dict(zipped)
                    data.append(data_dictionary)
            except:
                continue
    
    df = df.append(data,True)
    return df
print lifeCycle
print GTCycle
df = compareValues(GTCycle,lifeCycle)        
print df

   id2                                             deaths
0    1                                    [0, 28, 72, 80]
1    2                                            [0, 14]
2    3                                    [0, 13, 21, 31]
3    4                                            [0, 11]
4    5                                            [0, 13]
5    6                                             [0, 6]
6    7                                    [0, 14, 49, 65]
7    8                                            [0, 72]
8    9                            [0, 35, 36, 36, 91, 91]
9   10                            [0, 14, 60, 61, 67, 69]
10  11                                            [0, 39]
11  12                            [0, 33, 59, 63, 64, 79]
12  13                            [0, 18, 76, 91, 91, 91]
13  14                                            [0, 44]
14  15     [0, 2, 10, 18, 20, 23, 35, 42, 57, 91, 91, 91]
15  16                                    [0, 12, 91, 91]
16  17        

C:\Python27\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if __name__ == '__main__':


     id id2   deaths
0    10  36   [0, 4]
1    22  38   [0, 1]
2    50  17  [0, 91]
3    50  18  [0, 91]
4    50  25  [0, 91]
5    50  30  [0, 91]
6    50  35  [0, 91]
7    50  39  [0, 91]
8   186  17  [0, 91]
9   186  18  [0, 91]
10  186  25  [0, 91]
11  186  30  [0, 91]
12  186  35  [0, 91]
13  186  39  [0, 91]
14  189   5  [0, 13]


In [62]:
def evaluateGTMitosis(dataframe):
    mitosis = 0
    for index,row in dataframe.iterrows():
        if row.deaths[0] == 0:
            mitosis += len(row.deaths)/2-1
    print "There are %s mitosis from initial cells"%(mitosis)
    return mitosis
evaluateGTMitosis(GTCycle)

There are 92 mitosis from initial cells


92

In [63]:
def evaluateDetectedMitosis(dataframe):
    mitosis = 0
    for index,row in dataframe.iterrows():
        if row.deaths[0] == 0:            
            mitosis += len(row.deaths)/2-1
    print "There are %s mitosis well detected"%(mitosis)
    return mitosis
evaluateDetectedMitosis(df)

There are 0 mitosis well detected


0

In [ ]:
GTCycle.loc[GTCycle.id == 3].deaths.all()

In [ ]:
pg = programCycle(engine,"lifetmp")

print pg.loc[pg.myid%100 == 3]

In [79]:
GTCycle.loc[np.array(list(map(len,GTCycle.deaths.values)))>2]

,id,deaths
0,1,"[0, 30, 91, 91]"
2,12,"[0, 38, 91, 91]"
3,16,"[0, 22, 91, 91]"
9,25,"[0, 3, 48, 49, 91, 91, 91, 91]"
10,30,"[0, 1, 39, 44, 90, 91, 91, 91]"
12,41,"[0, 32, 91, 91]"
14,51,"[0, 17, 91, 91]"
15,55,"[0, 22, 75, 88, 91, 91, 91, 91]"
16,63,"[0, 21, 66, 81, 91, 91, 91, 91]"
17,76,"[0, 12, 50, 51, 91, 91, 91, 91]"


0    [0, 30, 91, 91]
1             [0, 4]
Name: deaths, dtype: object